In [22]:
import mmcv
import os
import os.path as osp
from mmcv import Config, load, dump
import functools
from functools import partial
from mmaction.models import build_model

In [23]:
def ls_conf(d):
    walks = list(os.walk(d))
    configs = [[osp.join(x[0], y) for y in x[2] if y.endswith('.py')] for x in walks]
    ret = []
    for x in configs:
        ret.extend(x)
    return ret

In [75]:
cfgs = ls_conf('configs/recognition/tsn')


In [76]:
meta = load('configs/recognition/tsn/metafile.yml')

In [3]:
metafiles = load('model_zoo.yml')

In [100]:
for item in metafiles['Import']:
    meta = load(item)
    for model in meta['Models']:
        if 'gpu_mem(M)' in model:
            model.pop('gpu_mem(M)')
        if 'iter time(s)' in model:
            model.pop('iter time(s)')
        if 'inference_time(video/s)' in model:
            model.pop('inference_time(video/s)')
        if 'gpus' in model['Metadata']:
            gpus = model['Metadata'].pop('gpus')
            model['Metadata']['Training Resources'] = f'{gpus} GPUs'
        if 'Training BatchSize / GPU' in model['Metadata']:
            batch = model['Metadata'].pop('Training BatchSize / GPU')
            model['Metadata']['Batch Size'] = batch
    dump(meta, item)

In [28]:
# Unify keys
keys = set()
meta_keys = set()
metric_keys = set()
for item in metafiles['Import']:
    meta = load(item)
    for model in meta['Models']:
        if 'config' in model:
            print(model)
        keys = keys.union(set(model.keys()))
        meta_keys = meta_keys.union(set(model['Metadata'].keys()))
        assert len(model['Results']) == 1
        try:
            metrics = model['Results'][0]['Metrics'].keys()
        except:
            print(item, model)
        metric_keys = metric_keys.union(set(metrics))

In [29]:
print(metric_keys)

{'mAP', 'mAP@0.3', 'top5 accuracy delta [w. RGB]', 'Top 1 Accuracy [w. RGB]', 'top1 10-view', 'mean Top 1 Accuracy', 'AP@0.95', 'AR@100', 'mAP@0.4', 'AUC', 'Top 1 Accuracy (efficient)', 'top5 accuracy [w. RGB]', 'top5 accuracy', 'AP@0.75', 'mAP@0.5', 'Top 1 Accuracy', 'AP@0.5', 'top5 accuracy (efficient)', 'Top 1 Accuracy delta [w. RGB]'}


In [27]:
for item in metafiles['Import']:
    meta = load(item)
    for model in meta['Models']:
        metrics = model['Results'][0]['Metrics']
        metrics = {k.replace('top1 accuracy', 'Top 1 Accuracy'): v for k, v in metrics.items()}
        metrics = {k.replace('top5 accuracy (accurate)', 'Top 5 Accuracy'): v for k, v in metrics.items()}
        metrics = {k.replace('mean top1 accuracy', 'Mean Top 1 Accuracy'): v for k, v in metrics.items()}
        model['Results'][0]['Metrics'] = metrics
    dump(meta, item)

In [112]:
keys

{'Config',
 'In Collection',
 'Metadata',
 'Modality',
 'Name',
 'Results',
 'Training Json Log',
 'Training Log',
 'Weights',
 'delta top1 acc (efficient/accurate)',
 'delta top5 acc (efficient/accurate)',
 'reference mAP@0.3',
 'reference mAP@0.4',
 'reference mAP@0.5',
 'reference top1 10-view',
 'reference top1 30-view',
 'reference top1 acc (efficient/accurate)',
 'reference top5 acc (efficient/accurate)',
 'refrence ckpt',
 'refrence json',
 'tag category'}

In [113]:
meta_keys

{'Architecture',
 'Batch Size',
 'Epochs',
 'Input',
 'Modality',
 'Parameters',
 'Pretrained',
 'Resolution',
 'Training Data',
 'Training Resources',
 'feature',
 'n_fft',
 'pseudo heatmap'}

In [90]:
meta['Models'][0]

{'Config': 'configs/recognition/tsn/tsn_r50_1x1x3_75e_ucf101_rgb.py',
 'In Collection': 'TSN',
 'Metadata': {'Architecture': 'ResNet50',
  'Epochs': 75,
  'Parameters': 23714981,
  'Training BatchSize / GPU': 32,
  'Training Data': 'UCF101',
  'gpus': 8,
  'pretrain': 'ImageNet'},
 'Modality': 'RGB',
 'Name': 'tsn_r50_1x1x3_75e_ucf101_rgb',
 'Results': [{'Dataset': 'UCF101',
   'Metrics': {'top1 acc': 83.03, 'top5 acc': 96.78},
   'Task': 'Action Recognition'}],
 'Training Json Log': 'https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_75e_ucf101_rgb/tsn_r50_1x1x3_75e_ucf101_rgb_20201023.json',
 'Training Log': 'https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_75e_ucf101_rgb/tsn_r50_1x1x3_75e_ucf101_rgb_20201023.log',
 'Weights': 'https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_75e_ucf101_rgb/tsn_r50_1x1x3_75e_ucf101_rgb_20201023-d85ab600.pth',
 'config': '[tsn_r50_1x1x3_75e_ucf101_rgb](/configs/recognition/tsn/tsn_r50_1x1x3_